In [ ]:
# Some setup
%load_ext autoreload

%autoreload 2

# Code name: Cocktail

A kmer tools box to use small kmer on long-reads.

- count kmer: pcon
- filter reads: kmerf
- correct reads: br
- polish assemblies: br

## Parameter

Some global variable usefull in this notebook don't hesitate to set it 

In [ ]:
SNAKEMAKE_CORE=8

## Dataset

### References

| dataset         | species            | info                          | genome size | 
|:----------------|:-------------------|:------------------------------|------------:|
| bacteria        | E. coli K12 MG1655 | NC_000913.3                   |      4.6 Mb |
| yeast           | S. cerevisiae      | NC_001133                     |     12.1Mb |
| metagenome      | Mock community     | [pacbio assembly](https://lomanlab.github.io/mockcommunity/)       |       42.1Mb |



### Reads

| dataset         | info                             | # bases (Gb) | coverage
|:----------------|:---------------------------------|:-------------|------------:|
| bacteria        | SRR10950300                      |              |         ??x |
| bacteria        | synthetic reads 90-99 identity % |              |         ??x |
| yeast           | Genoscope                        |              |         ??x |
| metagenome      | [R10 Native “3 Peaks” Data Release (2019-05-24)](https://lomanlab.github.io/mockcommunity/r10.html) |     | ??x |


## Prepare dataset

In [ ]:
!snakemake --cores $SNAKEMAKE_CORE --use-conda -p data_all -n

## Count

Compare pcon against kmc (3.1.1) and jellyfish (2.2.10) k in range(13, 19, 2)), on all dataset.

In [ ]:
!snakemake --cores $SNAKEMAKE_CORE --use-conda -p count_all -n

#### Benchmark

In [4]:
import altair
from cocktail import count

print(count.dataframe())

count.figure("bacteria")

     counter   dataset kmer_size  time  memory
0       pcon  bacteria        13    10      20
1       pcon  bacteria        15   100      40
2        kmc  bacteria        13    30      40
3  jellyfish  bacteria        13    14      25


alt.Chart(...)

## Filter

Evaluate effect of kmerf on dataset and on assembly. With different k and different ratio.

In [ ]:
!snakemake --cores $SNAKEMAKE_CORE --use-conda -p filter_all -n

### Memory usage

### CPU time

### Read quality

[FiltLong graph](https://github.com/rrwick/Filtlong#example-commands-detailed) 

### Assembly quality

## Correction

Compare br (with k in range(13, 19, 2)) against canu correction module (2.0), consent and necat on all dataset.

In [ ]:
!snakemake --cores $SNAKEMAKE_CORE --use-conda -p correct_all -n

### Memory usage

#### Simulate

#### Real

### CPU time

#### Simulate

#### Real

### Error rate

#### Simulate

#### Real

### Recall

#### Simulate

#### Real

### Precision

#### Simulate

#### Real

### Effect of coverage

One dataset ? br only ?

### Dipolid dataset

## Polish

Effect of polish on miniasm assembly on all dataset mode (with k in range (13, 19, 2) and abundance in range(10, 30))

In [ ]:
!snakemake --cores $SNAKEMAKE_CORE --use-conda -p polish_all -n

### Quast result